In [1]:
import numpy as np
import pandas as pd
import altair as alt
import geopandas as gpd
main_df=pd.read_csv('main_data.csv')
main_df = main_df.rename(columns={'Latitude':'latitude'})
main_df = main_df.rename(columns={'Longitude':'longitude'})
main_df.head()

,Country,id,Abbreviation,latitude,longitude,Density,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Region,iso_alpha_3
0,Afghanistan,4,AF,33.939110,67.709953,60,58.10%,"652,230","323,000",32.49,...,78.40%,0.28,38041754.0,48.9,9.30%,71.40%,11.12,"9,797,273",Asia,AFG
1,Albania,8,AL,41.153332,20.168331,105,43.10%,"28,748","9,000",11.78,...,56.90%,1.20,2854191.0,55.7,18.60%,36.60%,12.33,"1,747,593",Europe,ALB
2,Algeria,12,DZ,28.033886,1.659626,18,17.40%,"2,381,741","317,000",24.28,...,28.10%,1.72,43053054.0,41.2,37.20%,66.10%,11.70,"31,510,100",Africa,DZA
3,Andorra,20,AD,42.506285,1.521801,164,40.00%,468,NaN,7.20,...,36.40%,3.33,77142.0,NaN,NaN,NaN,NaN,"67,873",Europe,AND
4,Angola,24,AO,-11.202692,17.873887,26,47.50%,"1,246,700","117,000",40.73,...,33.40%,0.21,31825295.0,77.5,9.20%,49.10%,6.89,"21,061,025",Africa,AGO


In [2]:
#Visualising population , life expectancy and birth rate using scatter plot(birth rate and life expectancy)
# bar plot ( population and life expectancy)
# Chloropleth (Region)


In [3]:
region_color = alt.Color('Region:N', title='Region', scale=alt.Scale(scheme='tableau20'))
# Interactive selection for region
brush = alt.selection_interval()
conditional= alt.when(brush).then("Region").otherwise(alt.value('lightgray'))
connector= alt.selection_interval(fields=['Region'])

In [4]:
# Base scatter plot for GDP vs Population
scatter_life = alt.Chart(main_df).mark_circle(size=60).encode(
    x=alt.X('Life expectancy', title='Life expectancy', scale=alt.Scale(type='log')),
    y=alt.Y('Birth Rate', title='Birth Rate', scale=alt.Scale(type='log')),
    color=region_color,
    tooltip=['Country', 'Region', 'Life expectancy','Birth Rate'],
    opacity=alt.condition(brush, alt.value(1), alt.value(0.2))
).properties(
    title='Life Expectancy and Birth Rate',
    width=300,
    height=200
).add_params(
    brush
).interactive()

In [5]:
histogram_life = alt.Chart(main_df).mark_bar().encode(
    y=alt.Y('Population', title='Population'),
    x=alt.X('Life expectancy', title='life expectancy'),
    tooltip=['Country', 'Region', 'Life expectancy', 'Infant mortality'],
    color=conditional
).add_params(
    brush
).properties(
    title='population analysis',
    width=300,
    height=200
).interactive()



In [6]:
main_df = main_df.rename(columns={'Country':'name'})
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
country_shapes = gpd.read_file(url) # zipped shapefile
country_shapes = country_shapes[['NAME', 'CONTINENT', 'ISO_A3', 'geometry',]]
# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()
# Layering and configuring the components
basemap = (
alt.layer(
#sphere background
alt.Chart(sphere).mark_geoshape(fill='white'),
#add meridians and parallels (graticules)
alt.Chart(graticule).mark_geoshape(stroke='black'),
)
)
countries = alt.Chart(country_shapes).mark_geoshape(fill='Coral', stroke='white')
choropleth = (
alt.Chart(country_shapes)
.mark_geoshape().transform_lookup(
lookup='ISO_A3',
from_=alt.LookupData(data=main_df, key='iso_alpha_3', fields=['Region'])
)
.encode(color=conditional,
      opacity=alt.condition(brush, alt.value(1), alt.value(0.2))
).add_params(brush)
)
((basemap + countries + choropleth)
.project("mercator").properties(width=600, height=300)
)

C:\Users\Hp\AppData\Local\Programs\Python\Python39\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: HTTP response code on https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip: 0
  return ogr_read(


DataSourceError: '/vsizip/vsicurl/https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
# Combine the charts 
dashboard1 = alt.vconcat(
    choropleth,
    alt.hconcat(scatter_life, histogram_life)
).resolve_scale(
    color='shared'
) 
dashboard1.save("dashboard1.html")


